## <font color='blue'> Fine Tuning LLM open-sourse com Dataset Customizado Para Geração de Texto</font>

In [1]:
!pip install -q accelerate peft bitsandbytes transformers trl datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Imports
import accelerate  # Facilita o treinamento distribuído (multi-GPU, TPU, CPU) sem precisar reescrever o código
import peft        # Biblioteca para fine-tuning eficiente de modelos grandes (ex: LoRA, prefix tuning)
import bitsandbytes  # Permite usar quantização 8-bit ou 4-bit para reduzir o consumo de memória em modelos grandes
import transformers  # Principal biblioteca para trabalhar com modelos de linguagem (BERT, GPT, T5, etc.)
import trl          # Transformers Reinforcement Learning: usada para treinar modelos com técnicas de RL (ex: PPO, DPO, RLHF)
import datasets     # Carregamento e manipulação de datasets prontos ou personalizados (IMDB, AG News, etc.)


In [3]:
# Imports
import torch  # Biblioteca principal para computação com tensores e deep learning (base para modelos em PyTorch)

from datasets import load_dataset  # Função da Hugging Face para carregar datasets públicos ou personalizados

from transformers import (
    AutoModelForCausalLM,   # Carrega automaticamente um modelo de linguagem causal (ex: GPT)
    AutoTokenizer,          # Carrega automaticamente o tokenizer correspondente ao modelo
    BitsAndBytesConfig      # Configura quantização (ex: 8-bit, 4-bit) para reduzir uso de memória com modelos grandes
)

from transformers import TrainingArguments  # Define os argumentos de treinamento (batch size, epochs, etc.)

from peft import (
    AutoPeftModelForCausalLM,      # Carrega um modelo com PEFT (Parameter-Efficient Fine-Tuning) já aplicado
    LoraConfig,                    # Define a configuração do LoRA (Low-Rank Adaptation) para fine-tuning eficiente
    get_peft_model,                # Aplica LoRA ou outra técnica PEFT ao modelo
    prepare_model_for_kbit_training  # Prepara o modelo para treinamento com quantização (k-bit)
)

from trl import SFTConfig, SFTTrainer  # Trainer especializado para Supervised Fine-Tuning (SFT) com modelos da Hugging Face

import warnings  # Biblioteca padrão do Python para manipular alertas

warnings.filterwarnings('ignore')  # Ignora warnings (avisos) durante a execução para deixar a saída mais limpa

In [4]:
# Verifica o modelo da GPU
if torch.cuda.is_available():
    print("Número de GPUs:", torch.cuda.device_count())
    print("Modelo GPU:", torch.cuda.get_device_name(0))
    print('Total Memória [GB] da GPU:', torch.cuda.get_device_properties(0).total_memory/1e9)

Número de GPUs: 1
Modelo GPU: Tesla T4
Total Memória [GB] da GPU: 15.828320256


In [5]:
# Reset da memória da GPU(quando necessário)
from numba import cuda
device = cuda.get_current_device()
device.reset()

## Carregando o Dataset direto do HuggingFace
https://huggingface.co/datasets/nlpie/Llama2-MedTuned-Instructions

In [6]:
# Carrega dataset
dataset = load_dataset("nlpie/Llama2-MedTuned-Instructions")

README.md:   0%|          | 0.00/2.96k [00:00<?, ?B/s]

(…)-00000-of-00001-a8790d88efc2bc45.parquet:   0%|          | 0.00/91.1M [00:00<?, ?B/s]

(…)-00000-of-00001-b543c64b1786c03e.parquet:   0%|          | 0.00/6.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/200252 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/70066 [00:00<?, ? examples/s]

In [7]:
# Visualizar o dataset
dataset


DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'source'],
        num_rows: 200252
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output', 'source'],
        num_rows: 70066
    })
})

In [8]:
# Loop para visualizar os 3 primeiros registros

for i in range(3):
    data = dataset['train'][i]
    print(f'Data Point {i+1}:')
    print("Instructions:",data['instruction'])
    print("Input:",data['input'])
    print("Output:",data['output'])
    print('-'*100)

Data Point 1:
Instructions: In your role as a medical professional, address the user's medical questions and concerns.
Input: My relative suffering from secondary lever cancer ( 4th stage as per Allopathic doctor) and primary is in rectum. He is continuously with 103 to 104 degree F fever. Allpathic doctor suggested chemo only after fever subsidises. Is treatment possible at Lavanya & what is the time scale of recover.
Output: Hi, dairy have gone through your question. I can understand your concern. He has rectal cancer with liver metastasis. It is stage 4 cancer. Surgery is not possible at this stage. Only treatment options are chemotherapy and radiotherapy according to type of cancer. Inspite of all treatment prognosis is poor. Life expectancy is not good. Consult your doctor and plan accordingly. Hope I have answered your question, if you have any doubts then contact me at bit.ly/ Chat Doctor. Thanks for using Chat Doctor. Wish you a very good health.
-------------------------------

## Trabalhando com amostras menores (para acelerar o tempo de treino)

In [9]:
# Amostra de treino
dataset["train"]= dataset["train"].select(range(4000))

In [10]:
# Amostra teste
dataset["test"]= dataset["train"].select(range(350))

In [11]:
# Visualiza o dataset
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'source'],
        num_rows: 4000
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output', 'source'],
        num_rows: 70066
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'source'],
        num_rows: 350
    })
})

## Ajustando o formato do prompt de entrada para o LLM


In [12]:
# Função para criação de prompt

def cria_prompt(sample):
    prompt= sample['instruction']
    prompt+= sample['input']
    single_turn_prompt = f"Instruction: {prompt}<|end_of_turn|>AI assistant: {sample['output']}"
    return single_turn_prompt

In [13]:
# Exemplo de uso da função
cria_prompt(dataset["train"][0])

"Instruction: In your role as a medical professional, address the user's medical questions and concerns.My relative suffering from secondary lever cancer ( 4th stage as per Allopathic doctor) and primary is in rectum. He is continuously with 103 to 104 degree F fever. Allpathic doctor suggested chemo only after fever subsidises. Is treatment possible at Lavanya & what is the time scale of recover.<|end_of_turn|>AI assistant: Hi, dairy have gone through your question. I can understand your concern. He has rectal cancer with liver metastasis. It is stage 4 cancer. Surgery is not possible at this stage. Only treatment options are chemotherapy and radiotherapy according to type of cancer. Inspite of all treatment prognosis is poor. Life expectancy is not good. Consult your doctor and plan accordingly. Hope I have answered your question, if you have any doubts then contact me at bit.ly/ Chat Doctor. Thanks for using Chat Doctor. Wish you a very good health."

In [14]:
# Exemplo de uso da função
cria_prompt(dataset["train"][6])

"Instruction: Given your profession as a doctor, please provide responses to the medical questions using the patient's description.\nAnalyze the question given its context. Give both long answer and yes/no decision.###Question: Are serum pepsinogens and Helicobacter pylori associated with esophageal squamous cell carcinoma in a high-risk area in China?\n###Context: The role of serum pepsinogen level and Helicobacter pylori infection in esophageal carcinoma remains controversial. It may be a risk or protective factor, or without association with esophageal carcinoma. We prospectively examined associations between serum pepsinogen status, H pylori infection and the risk of esophageal squamous cell carcinoma in the Chinese population. In the present study, 1501 subjects from a community-based general population of Northern China were included. The incidence of esophageal carcinoma among the subjects was registered during a 15-year follow-up period by annual home visit, and the risks of lo

## Parâmetros de Quantização

Quantização de LLM (Large Language Model) é um processo de reduzir a precisão dos parâmetros de um modelo de linguagem grande (LLM) para um valor mais baixo, sem perder muito da sua performance. Isso é feito para reduzir a complexidade do modelo e, consequentemente, melhorar a eficiência computacional e a eficiência de armazenamento.

A quantização de LLM é útil em várias situações, como:

- Redução do tamanho do modelo: a quantização pode reduzir o tamanho do modelo, o que pode melhorar a eficiência de armazenamento e transferência de dados.

- Melhoria da eficiência computacional: a quantização pode melhorar a eficiência computacional do modelo, tornando-o mais rápido e mais eficiente em termos de recursos computacionais.

- Aumento da escalabilidade: a quantização pode permitir que os modelos sejam treinados em hardware mais barato e mais eficiente, o que pode ser útil em aplicações de produção.

BitsAndBytesConfig é usado para configurar a quantização de LLM. Ele é um objeto que contém informações sobre a quantização, como:

- Tipo de quantização: o tipo de quantização a ser usado, como quantização de ponto flutuante ou quantização de inteiro.

- Número de bits: o número de bits a ser usado para representar os parâmetros do modelo.

O BitsAndBytesConfig é usado para configurar a quantização de LLM e pode ser usado em conjunto com outros parâmetros para treinar um modelo de linguagem mais eficiente e escalável.

In [15]:
# Define os Parâmetros da quantização
bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype="float16",
                                bnb_4bit_use_double_quant=True)

## Carregando o LLM e o Tokenizador
https://huggingface.co/berkeley-nest/Starling-LM-7B-alpha

In [16]:
# Nome do repositório
repositorio_hf = "berkeley-nest/Starling-LM-7B-alpha"

In [17]:
# Carrega o LLM aplicando a quantização
modelo_llm = AutoModelForCausalLM.from_pretrained(repositorio_hf,
                                                  quantization_config=bnb_config,
                                                  device_map="auto", # usa GPU automaticamente se possível
                                                  use_cache=False)  # Desativa a cache para evitar vazamentos ou conflitos

config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [18]:
# Carrega o tokenizador do LLM
tokenizer = AutoTokenizer.from_pretrained(repositorio_hf)

tokenizer_config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

In [19]:
# Define o token de fim da sentença
tokenizer.pad_token = tokenizer.eos_token


In [20]:
# Define a direção do padding
tokenizer.padding_side = "right"

## Gerando resposta com o LLM Antes do ajuste fino

In [21]:
# Função para gerar reposta do llm

def gera_resposta_antes_fine_tuning(prompt,model):
    # aplica o tokenizador
    encoded_input = tokenizer(prompt,
                              return_tensors='pt',
                              add_special_tokens=True)

    # tranforma o input em tensor
    model_inputs = encoded_input.to('cuda')

    # Gera a resposta
    generated_ids = model.generate(**model_inputs,
                                   max_new_tokens=1024,
                                   do_sample=True,
                                   pad_token_id=tokenizer.eos_token_id)

    # Decodifica a resposta
    decoded_output = tokenizer.batch_decode(generated_ids)

    return decoded_output[0].replace(prompt,"")

In [22]:
# Exemplo de prompt
prompt = """Instruction: Your goal is to determine the relationship between the two provided clinical sentences and classify them into one of the following categories:
Contradiction: If the two sentences contradict each other. Neutral: If the two sentences are unrelated to each other. Entailment: If one of the sentences logically entails the other. """
prompt += '''Sentence 1: For his hypotension, autonomic testing confirmed orthostatic hypotension. Sentence 2: the patient has orthostatic hypotension <|end_of_turn|>'''
prompt += "AI Assistant:"

In [23]:
print(prompt)

Instruction: Your goal is to determine the relationship between the two provided clinical sentences and classify them into one of the following categories:
Contradiction: If the two sentences contradict each other. Neutral: If the two sentences are unrelated to each other. Entailment: If one of the sentences logically entails the other. Sentence 1: For his hypotension, autonomic testing confirmed orthostatic hypotension. Sentence 2: the patient has orthostatic hypotension <|end_of_turn|>AI Assistant:


In [24]:
# Gera resposta sem fine tuning
gera_resposta_antes_fine_tuning(prompt,modelo_llm)

'<s> Instruction: Your goal is to determine the relationship between the two provided clinical sentences and classify them into one of the following categories:\nContradiction: If the two sentences contradict each other. Neutral: If the two sentences are unrelated to each other. Entailment: If one of the sentences logically entails the other. Sentence 1: For his hypotension, autonomic testing confirmed orthostatic hypotension. Sentence 2: the patient has orthostatic hypotension <|end_of_turn|> AI Assistant: Entailment<|end_of_turn|>'

## Parâmetro LoRa para PEFT

In [25]:
# Define os parâmetros Lora
peft_config = LoraConfig(r=8,
                         lora_alpha=16,
                         lora_dropout=0.05,
                         bias="none",
                         task_type="CAUSAL_LM")

Aqui está a descrição de cada parâmetro na configuração LoraConfig:

**r**: Especifica o tamanho do fator de expansão utilizado no mecanismo LoRA (Low-Rank Adaptation). Um valor de r = 8 indica que a adaptação de baixo posto aumenta a dimensionalidade das projeções em um fator de 8. Essencialmente, este parâmetro controla o tamanho do aumento de dimensionalidade que é aplicado às camadas selecionadas do modelo para permitir uma maior capacidade de adaptação sem alterar significativamente a quantidade total de parâmetros do modelo.

**lora_alpha**: Multiplicador para o fator de aprendizado aplicado especificamente aos parâmetros de adaptação LoRA. Com lora_alpha = 16, o aprendizado desses parâmetros específicos é ampliado em 16 vezes em comparação com o restante dos parâmetros do modelo. Isso permite que os parâmetros de adaptação se ajustem mais rapidamente durante o treinamento.

**lora_dropout**: Taxa de dropout aplicada nas modificações de LoRA. O valor de 0.05 significa que 5% dos elementos nas projeções LoRA serão zerados aleatoriamente durante o treinamento. O dropout é uma técnica de regularização usada para evitar o sobreajuste, forçando o modelo a aprender representações mais robustas, pois não pode depender de qualquer conexão única entre os neurônios.

**bias**: Configura se os termos de viés (bias) são usados ou não nos ajustes LoRA. Aqui, está definido como "none", indicando que não serão adicionados termos de viés nas adaptações LoRA. Escolher não usar viés pode simplificar o modelo e focar as adaptações unicamente nos pesos.

**task_type**: Define o tipo de tarefa para a qual o modelo está sendo configurado. "CAUSAL_LM" refere-se a um modelo de linguagem causal, onde a previsão de cada palavra subsequente depende apenas das palavras anteriores e não de qualquer palavra futura. Isso é típico de modelos que geram texto de maneira sequencial, como a geração de texto ou a modelagem de diálogo.

Esses parâmetros são configurados para ajustar o modelo de maneira eficaz e específica, aproveitando a técnica de adaptação de baixo posto para realizar ajustes finos no modelo sem o custo computacional de re-treinar todos os parâmetros do modelo original.

In [26]:
# PREPARA O MODELO PARA AJUSTE Fino
modelo_llm=prepare_model_for_kbit_training(modelo_llm)

In [27]:
# Concatena o modelo base com os parâmetros lora
# get_peft_model(modelo base do hugging face, acrescenta uma camada)---> esta camada que será treinada
modelo_llm = get_peft_model(modelo_llm,peft_config)

### Argumentos de treino


In [28]:
# Define os argumentos de treino do modelo
# Argumentos de treino são os hiperparâmetros

training_arguments = SFTConfig(  ##
        output_dir="modelo_ajustado",
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=2,
        max_steps=250,
        fp16=True,
        packing=False,  ##
        max_seq_length=512,  ##
        dataset_text_field="text",  ##
        push_to_hub=True
)


Os parâmetros do objeto TrainingArguments são usados para configurar o processo de treinamento de um modelo de aprendizado de máquina. Aqui está a descrição de cada um deles:

**output_dir**: Diretório onde os artefatos do modelo treinado, como os pesos do modelo e as configurações, serão salvos. Aqui, está definido como "modelo_ajustado".

**per_device_train_batch_size**: Número de exemplos processados simultaneamente em cada dispositivo (como uma GPU) durante o treinamento.

**gradient_accumulation_steps**: Número de passos de treinamento para acumular os gradientes antes de realizar uma atualização dos parâmetros. Isso permite usar um tamanho de lote efetivo maior do que o que cabe na memória do dispositivo.

**optim**: Otimizador usado para atualizar os pesos do modelo. "paged_adamw_32bit" é uma variação do otimizador AdamW que otimiza o uso de memória utilizando precisão de 32 bits.

**learning_rate**: Taxa de aprendizado inicial usada pelo otimizador.

**lr_scheduler_type**: Tipo de agendador de taxa de aprendizado, que ajusta a taxa de aprendizado baseado no número de épocas ou passos. "cosine" indica que a taxa de aprendizado segue uma curva cosseno, diminuindo gradualmente ao longo das épocas.

**save_strategy**: Estratégia para salvar o modelo durante o treinamento. "epoch" significa que o modelo será salvo ao final de cada época.

**logging_steps**: Número de passos de treinamento após os quais os logs serão gravados. Aqui, está definido como 10, significando que o progresso do treinamento será logado a cada 10 passos.

**num_train_epochs**: Número total de épocas de treinamento. Uma época é uma passagem completa pelos dados de treinamento.

**max_steps**: Número máximo de passos de treinamento a serem executados. Útil para terminar o treinamento após um número fixo de passos, independentemente das épocas.

**fp16**: Se ativado, permite o uso de ponto flutuante de 16 bits durante o treinamento para reduzir o uso de memória e possivelmente acelerar a computação. Aqui, está definido como True, indicando que está ativado.

## Parâmetros do Supervised Fine-Tuning Trainer(SFTT)

In [36]:
from huggingface_hub import notebook_login
notebook_login()


In [37]:
trainer = SFTTrainer(
        model=modelo_llm,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        peft_config=peft_config,
        args=training_arguments,
        processing_class=tokenizer,
        formatting_func=cria_prompt
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Aqui está a descrição de cada parâmetro utilizado na criação de uma instância de SFTTrainer:

**model**: Define o modelo de aprendizado de máquina que será treinado.

**peft_config**: Configuração para o treinamento eficiente de parâmetros do modelo (PEFT).

**max_seq_length**: Comprimento máximo de sequência que o modelo pode aceitar. Está definido como 512, o que significa que cada entrada (como um texto) será truncada ou preenchida até esse tamanho para manter um tamanho consistente nas entradas.

**tokenizer**: O tokenizer é usado para converter texto em um formato que o modelo pode processar (como tokens ou números). Aqui, é usado um tokenizer específico, que deve ser compatível com o modelo escolhido.

**packing**: Um parâmetro booleano que, quando verdadeiro (True), indica que o treinador deve tentar empacotar eficientemente as sequências de entrada para otimizar o uso da memória e o desempenho do treinamento.

**formatting_func**: Função utilizada para formatar os dados de entrada antes de serem fornecidos ao modelo.

**args**: Parâmetros adicionais de configuração do treinamento, agrupados em dsa_training_arguments. Esses argumentos definem diversas configurações como taxa de aprendizado, estratégia de salvamento, entre outros, que foram descritos anteriormente.

**train_dataset**: Conjunto de dados utilizado para treinar o modelo.

**eval_dataset**: Conjunto de dados utilizado para avaliar o modelo.

## Treinamento(Ajuste fino) do LLM

In [38]:
%%time
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pablorachid23 (pablorachid23-out-of-market) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,2.079700
20,1.799900
30,1.589900
40,1.394600
50,1.156400
60,1.377800
70,1.643300
80,1.092500
90,1.062700
100,1.147600


CPU times: user 17min 9s, sys: 4min 10s, total: 21min 19s
Wall time: 22min 46s


TrainOutput(global_step=250, training_loss=1.3294880447387696, metrics={'train_runtime': 1362.8743, 'train_samples_per_second': 0.734, 'train_steps_per_second': 0.183, 'total_flos': 1.16355695450112e+16, 'train_loss': 1.3294880447387696})

In [39]:
# Salva o modelo
trainer.save_model("modelo_ajustado")

wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


In [40]:
# Descarrega o modelo ajustado
modelo_final = modelo_llm.merge_and_unload()

In [41]:
# Função para gerar reposta do llm

def gera_resposta_depois_fine_tuning(prompt,model):
    # aplica o tokenizador
    encoded_input = tokenizer(prompt,
                              return_tensors='pt',
                              add_special_tokens=True)

    # tranforma o input em tensor
    model_inputs = encoded_input.to('cuda')

    # Gera a resposta
    generated_ids = model.generate(**model_inputs,
                                   max_new_tokens=512,
                                   do_sample=True,
                                   use_cache=False,
                                   pad_token_id=tokenizer.eos_token_id)

    # Decodifica a resposta
    decoded_output = tokenizer.batch_decode(generated_ids)

    return decoded_output[0]

### Inferência 1

In [42]:
%%time
prompt = "Instruction: In your role as a medical professional, address the user's medical questions and concerns. "
prompt += "I have a white tab under my tounge that is not only painful when i touch it but bleeds as well. not sure what it is, or why I got it. Can you give me any advise? <|end_of_turn|> "
prompt += "AI Assistant:"
response = gera_resposta_depois_fine_tuning(prompt, modelo_final)
print(response)

<s> Instruction: In your role as a medical professional, address the user's medical questions and concerns. I have a white tab under my tounge that is not only painful when i touch it but bleeds as well. not sure what it is, or why I got it. Can you give me any advise? <|end_of_turn|>  AI Assistant: There are several possible causes for a white tab underneath your tongue that is causing pain and bleeding. Here are a few possibilities:

1. Oral thrush: This is a fungal infection caused by the overgrowth of yeast (Candida) in the mouth. It can cause white spots or plaques on the inner lining of the mouth, and can sometimes cause pain and discomfort.

2. Geographic tongue: This is a condition in which the surface of the tongue has irregular, smooth white patches that seem to take on a map-like appearance. While usually painless, geographic tongue can sometimes be associated with pain and tenderness of the tongue.

3. Trauma: If you have injured your tongue in any way, such as biting it or

### Inferência 2

In [43]:
%%time
prompt = "Instruction: In your capacity as a healthcare expert, offer insights and recommendations in response to users' medical inquiries. "
prompt += "I have terrible anxiety and depression. I've tried various therapists and pills, but nothing's helped. <|end_of_turn|> "
prompt += "AI Assistant:"
response = gera_resposta_depois_fine_tuning(prompt, modelo_final)
print(response)

<s> Instruction: In your capacity as a healthcare expert, offer insights and recommendations in response to users' medical inquiries. I have terrible anxiety and depression. I've tried various therapists and pills, but nothing's helped. <|end_of_turn|>  AI Assistant: I'm sorry to hear about your struggles with anxiety and depression. It's important to understand that while both conditions can co-occur, they are separate disorders and often require different types of treatment.

For anxiety, therapy can be helpful, but it's also important to consider medication. In addition to the antidepressants you may be taking, there are anti-anxiety medications that can be helpful. For example, benzodiazepines like Xanax or Valium can provide immediate relief of anxiety symptoms, but they can lead to dependence, especially with long-term use. Selective serotonin reuptake inhibitors (SSRIs) like Prozac or Zoloft can help prevent anxiety and depression, and they have fewer side effects. Non-SSRIs lik

### Inferência 3

In [44]:
%%time
prompt = "Instruction: As a medical chatbot, your responsibility is to provide information and guidance on medical matters to users. "
prompt += "Hi sir, I am so happy with this website. First of all thanks for giving this opportunity. I am the  Software employee.My age is 24. My height is 169cm .Recently I got back pain and some pain in chest. How can i get relief from those pains.How i improve my health and which type of diseases will attack to my life in future. Please give Some health tips for heart and kidneys protection. <|end_of_turn|> "
prompt += "AI Assistant:"
response = gera_resposta_depois_fine_tuning(prompt, modelo_final)
print(response)

<s> Instruction: As a medical chatbot, your responsibility is to provide information and guidance on medical matters to users. Hi sir, I am so happy with this website. First of all thanks for giving this opportunity. I am the  Software employee.My age is 24. My height is 169cm .Recently I got back pain and some pain in chest. How can i get relief from those pains.How i improve my health and which type of diseases will attack to my life in future. Please give Some health tips for heart and kidneys protection. <|end_of_turn|>  AI Assistant: Hello, thank you for your kind words. As a medical chatbot, I cannot diagnose or treat specific conditions, but I can provide you with some general advice.

For back pain, I suggest you try the RICE method: Rest, Ice, Compression, and Elevation. This might help reduce inflammation and discomfort. For chest pain, it's essential to seek medical attention immediately to determine the cause. You could also try slow-deep breathing, leaning forward, and pla